<a href="https://colab.research.google.com/github/UtkarshRasal/deep_learning/blob/master/neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch workflow fundamentals